#### Time(t), Voltage(V), Load Torque(LT), Current(I), Speed(w)
Control variables - V, LT  
State variables - I, w

### Read data

In [28]:
t_file = []
voltage = []
loadTorque = []
current = []
speed = []


f = open('DC_motor_001.txt')
for line in f:
    values = line.split()
    t_file.append(float(values[0]))
    voltage.append(float(values[1]))
    loadTorque.append(float(values[2]))
    current.append(float(values[3]))
    speed.append(float(values[4]))
    

#### Test print

In [ ]:
t_file

### Fit sparse and non-sparse model

In [146]:
import numpy as np
import pysindy as ps

optNonSparse = ps.STLSQ(threshold=0, fit_intercept=True)
optSparse = ps.STLSQ(threshold=10.0, fit_intercept=True)

X = np.column_stack((current, speed))
U = np.column_stack((voltage, loadTorque))
t = np.array(t_file)
featureNames = ['i', 'w', 'V', 'LT']


varsLib = ps.PolynomialLibrary(degree=2, include_interaction=False)
controlLib = ps.PolynomialLibrary(degree=1, include_interaction=False)
inputs_temp = [[0,1,1,1],
                [2,3,3,3]]
inputs_per_library = np.asarray(inputs_temp)

featureLibrary = ps.GeneralizedLibrary(
    [varsLib, controlLib],
    inputs_per_library = inputs_per_library
)

modelNonSparse = ps.SINDy(feature_names=featureNames, optimizer=optNonSparse,feature_library=featureLibrary)
modelNonSparse.fit(X, t=t, u=U)

modelSparse = ps.SINDy(feature_names=featureNames, optimizer=optSparse, feature_library=featureLibrary)
modelSparse.fit(X, t=t, u=U)


c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.15339e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.15339e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.15339e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
c:\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.15339e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


SINDy(differentiation_method=FiniteDifference(),
      feature_library=<pysindy.feature_library.generalized_library.GeneralizedLibrary object at 0x0000018BBE739690>,
      feature_names=['i', 'w', 'V', 'LT'],
      optimizer=STLSQ(fit_intercept=True, threshold=10.0))

### Sparse Model

In [147]:
modelSparse.print()

(i)' = -309.130 i + -31.005 w + 618.742 V + 55.266
(w)' = 199.473 i + -3952.991 LT + -317.738


### Non-Sparse Model

In [142]:
modelNonSparse.print()

(i)' = 31.094 1 + -309.223 i + -31.003 w + -0.002 i^2 + 31.094 1 + 618.736 V + -6.582 LT
(w)' = -0.393 1 + 204.968 i + 0.119 w + -0.393 1 + -10.428 V + -3986.849 LT
